
# California Housing — Simple & Multiple Linear Regression

Author: shobhit pandit 
Date: 28-8-2025  

## 1. Overview
This notebook predicts median house value for California districts using district-level features.
We build and compare:
- Simple Linear Regression (SLR) — using the single best predictor
- Multiple Linear Regression (MLR) — using all relevant predictors via a preprocessing pipeline

We cover complete EDA, preprocessing, model development, evaluation, and final model selection.



## 2. Problem Statement
Objective:Predict `median_house_value` using features such as income, rooms, bedrooms, population, households, housing age, latitude/longitude, and `ocean_proximity` (categorical).  
We will:
1. Explore & visualize the data.
2. Preprocess (handle missing values, encode categoricals, scale numeric features if needed).
3. Train SLR & MLR models.
4. Evaluate with regression metrics: MSE, RMSE, MAE, R².
5. Compare models for a balance of accuracy and interpretability.

In [2]:

# === Paths ===
from pathlib import Path
DATA_PATH = Path("Data_file.xlsx")   
TARGET_COL = "median_house_value"



## 4. Setup


In [3]:

# Core
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Plotting (matplotlib only, no seaborn)
import matplotlib.pyplot as plt

# Modeling
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Saving model
import joblib

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)



## 5. Load Data


In [6]:
if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Could not find {DATA_PATH}"
    )

df = pd.read_excel(DATA_PATH)
print("Shape:", df.shape)
df.head()


FileNotFoundError: Could not find Data_file.xlsx


## 6. Quick Data Overview


In [ ]:

# Basic info
print("Columns:", list(df.columns))
print("\nData types:\n", df.dtypes)
print("\nMissing values:\n", df.isna().sum())

# Basic descriptive stats for numeric columns
df.describe().T



## 7. Exploratory Data Analysis (EDA)
### 7.1 Target Distribution


In [ ]:

plt.figure()
df[TARGET_COL].hist(bins=50)
plt.title("Distribution of Target: " + TARGET_COL)
plt.xlabel(TARGET_COL)
plt.ylabel("Frequency")
plt.show()



### 7.2 Numeric Feature Histograms


In [ ]:

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET_COL in numeric_cols:
    numeric_cols.remove(TARGET_COL)

for col in numeric_cols:
    plt.figure()
    df[col].hist(bins=50)
    plt.title(f"Histogram: {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()



### 7.3 Correlation Heatmap (Numeric)


In [ ]:

corr = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest', aspect='auto')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.title("Correlation Heatmap (Numeric)")
plt.colorbar()
plt.tight_layout()
plt.show()

corr[TARGET_COL].sort_values(ascending=False)



### 7.4 Target vs. Individual Predictors


In [ ]:

top_corr = corr[TARGET_COL].drop(labels=[TARGET_COL]).abs().sort_values(ascending=False)
top_features = top_corr.head(6).index.tolist()

for col in top_features:
    plt.figure()
    plt.scatter(df[col], df[TARGET_COL], alpha=0.3)
    plt.title(f"{TARGET_COL} vs {col}")
    plt.xlabel(col)
    plt.ylabel(TARGET_COL)
    plt.show()



## 8. Data Preprocessing
- Impute missing numeric values with **median**.
- One-hot encode **categorical** variables (e.g., `ocean_proximity`).
- Optionally, **scale** numeric features (useful for some models; linear regression is scale-invariant for OLS, but scaling can help numeric stability).


In [ ]:

# Identify features
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop=None, sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

len(X_train), len(X_test)



## 9. Simple Linear Regression (SLR)
We choose the **single most correlated numeric feature** with the target and fit a univariate linear regression.


In [ ]:

# Select best single numeric predictor by absolute correlation
num_corr = df[numeric_features + [TARGET_COL]].corr()[TARGET_COL].drop(labels=[TARGET_COL])
best_feature = num_corr.abs().sort_values(ascending=False).index[0]
print("Best single predictor:", best_feature)

X_train_slr = X_train[[best_feature]].copy()
X_test_slr  = X_test[[best_feature]].copy()

slr = LinearRegression()
slr.fit(X_train_slr, y_train)

y_pred_slr = slr.predict(X_test_slr)

mse_slr = mean_squared_error(y_test, y_pred_slr)
rmse_slr = np.sqrt(mse_slr)
mae_slr = mean_absolute_error(y_test, y_pred_slr)
r2_slr = r2_score(y_test, y_pred_slr)

print(f"SLR — Test MSE: {mse_slr:.2f}, RMSE: {rmse_slr:.2f}, MAE: {mae_slr:.2f}, R²: {r2_slr:.4f}")


In [ ]:

# Plot: Actual vs Predicted (SLR)
plt.figure()
plt.scatter(y_test, y_pred_slr, alpha=0.3)
plt.title("SLR — Actual vs Predicted")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
plt.show()

# Residuals
resid_slr = y_test - y_pred_slr
plt.figure()
plt.scatter(y_pred_slr, resid_slr, alpha=0.3)
plt.axhline(0)
plt.title("SLR — Residuals vs Fitted")
plt.xlabel("Fitted")
plt.ylabel("Residuals")
plt.show()



## 10. Multiple Linear Regression (MLR)
We use all relevant features via a preprocessing pipeline (imputation, one-hot encoding, scaling) with OLS LinearRegression.


In [25]:

mlr = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LinearRegression())
])

mlr.fit(X_train, y_train)
y_pred_mlr = mlr.predict(X_test)

mse_mlr = mean_squared_error(y_test, y_pred_mlr)
rmse_mlr = np.sqrt(mse_mlr)
mae_mlr = mean_absolute_error(y_test, y_pred_mlr)
r2_mlr = r2_score(y_test, y_pred_mlr)

print(f"MLR — Test MSE: {mse_mlr:.2f}, RMSE: {rmse_mlr:.2f}, MAE: {mae_mlr:.2f}, R²: {r2_mlr:.4f}")


MLR — Test MSE: 4908290571.35, RMSE: 70059.19, MAE: 50670.49, R²: 0.6254


In [ ]:

# Cross-validation on training data (for more robust score estimation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2_scores = cross_val_score(mlr, X_train, y_train, scoring="r2", cv=cv)
cv_rmse_scores = np.sqrt(-cross_val_score(mlr, X_train, y_train, scoring="neg_mean_squared_error", cv=cv))

print("MLR — CV R² (mean ± std): {:.4f} ± {:.4f}".format(cv_r2_scores.mean(), cv_r2_scores.std()))
print("MLR — CV RMSE (mean ± std): {:.2f} ± {:.2f}".format(cv_rmse_scores.mean(), cv_rmse_scores.std()))


In [ ]:

# Plot: Actual vs Predicted (MLR)
plt.figure()
plt.scatter(y_test, y_pred_mlr, alpha=0.3)
plt.title("MLR — Actual vs Predicted")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
plt.show()

# Residuals
resid_mlr = y_test - y_pred_mlr
plt.figure()
plt.scatter(y_pred_mlr, resid_mlr, alpha=0.3)
plt.axhline(0)
plt.title("MLR — Residuals vs Fitted")
plt.xlabel("Fitted")
plt.ylabel("Residuals")
plt.show()



## 11. Model Comparison


In [ ]:

results = pd.DataFrame({
    "Model": ["Simple Linear Regression", "Multiple Linear Regression"],
    "MSE": [mse_slr, mse_mlr],
    "RMSE": [np.sqrt(mse_slr), np.sqrt(mse_mlr)],
    "MAE": [mae_slr, mae_mlr],
    "R2": [r2_slr, r2_mlr]
}).sort_values(by="RMSE")

results



## 12. Interpreting the MLR (Feature Effects)
We extract the linear model coefficients with their transformed feature names.


In [ ]:

# Extract feature names after preprocessing
ct = mlr.named_steps["preprocess"]
ohe = None
for name, trans, cols in ct.transformers_:
    if name == "cat":
        ohe = trans.named_steps["onehot"]
        cat_cols = list(cols)
    elif name == "num":
        num_cols = list(cols)

feature_names = []
# Numeric features (after scaler)
feature_names.extend(num_cols)

# One-hot-encoded categorical features
if ohe is not None:
    cat_feature_names = ohe.get_feature_names_out(cat_cols).tolist()
    feature_names.extend(cat_feature_names)

# Coefficients
coefs = mlr.named_steps["model"].coef_
coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs}).sort_values(by="coef", key=lambda s: s.abs(), ascending=False)
coef_df.head(20)



## 13. Save Final Model


In [ ]:

# Save the trained MLR pipeline
MODEL_PATH = Path("final_mlr_pipeline.joblib")
joblib.dump(mlr, MODEL_PATH)
MODEL_PATH



## 14. Using the Saved Model
```python
import joblib
import pandas as pd

pipe = joblib.load("final_mlr_pipeline.joblib")

# Example: predict for new rows (must match training columns, including 'ocean_proximity' if used)
new_data = pd.DataFrame([{
    "longitude": -122.23,
    "latitude": 37.88,
    "housing_median_age": 41.0,
    "total_rooms": 880.0,
    "total_bedrooms": 129.0,
    "population": 322.0,
    "households": 126.0,
    "median_income": 8.3252,
    "ocean_proximity": "NEAR BAY"
}])

pipe.predict(new_data)
```
